In [222]:
import requests
import json
import re
import pandas as pd 
import numpy as np
import time

In [434]:
df = pd.read_csv('data/top_10000_ingredients.csv')

# df = df.iloc[9428:]
# df.loc[df['ingredients']=='slightly beaten eggs']
# df = df.iloc[9356:9358]
# df

,ingredients,recipe_counts


In [358]:
api_key = "6QckmmkUv58RO4eu9eiXKUD1a0L60mSjTnHfa036"
def search_usda_food_with_fdcid(fdcid):
    '''
    given fdc id, return the detailed info
    '''
    url = "https://api.nal.usda.gov/fdc/v1/food/{}?api_key={}".format(fdcid, api_key)
    response = requests.request("GET", url)

    if response.status_code != 200:
        return ''
    
    return response.json()
def search_fdcid(ingredient):
    '''
    Return the fdc id of the search for the first one with food portions
    '''
    url = "https://api.nal.usda.gov/fdc/v1/foods/search?query={}&model=foodPortion&api_key=6QckmmkUv58RO4eu9eiXKUD1a0L60mSjTnHfa036"\
          .format(ingredient)
    response = requests.request("GET", url)

    if response.status_code != 200:
        return None
    
    for food in response.json().get("foods"):
        fdc_id = food.get("fdcId")
        json_data = search_usda_food_with_fdcid(fdc_id)
        if isinstance(json_data, str):
            continue
#         print(fdc_id)
#         print(search_usda_food_with_fdcid(fdc_id).get("foodPortions"))
        if( json_data.get("foodPortions") !=None ):
            if( len(json_data.get("foodPortions")) > 0 ):
                return fdc_id
    return None


In [435]:
import time

ingredient_density={}
start = time.time()
ing_name=[]
standard_vol=[]
standard_unit=[]
standard_weight=[]
for index, row in df.iterrows():
    name = row.ingredients
    fdcid = search_fdcid(row.ingredients)
    if fdcid == None:
        json_data={"foodPortions":[{}]}
    else:
        json_data = search_usda_food_with_fdcid(fdcid)

        if isinstance(json_data, str):
            continue
        elif 'portionDescription' in json_data.get("foodPortions")[0].keys():
            ing_amt = json_data.get("foodPortions")[0]['portionDescription'].split(' ')[0]
            if ing_amt.isdigit():
                standard_vol.append(int(ing_amt))
                standard_unit.append(json_data.get("foodPortions")[0]['portionDescription'].split(' ')[1])
                standard_weight.append(json_data.get("foodPortions")[0]['gramWeight'])
                ing_name.append(name)                  
        elif ('amount' in json_data.get("foodPortions")[0].keys()) and ('modifier' in json_data.get("foodPortions")[0].keys()):
            standard_vol.append(json_data.get("foodPortions")[0]["amount"])
            standard_unit.append(json_data.get("foodPortions")[0]["modifier"])
            standard_weight.append(json_data.get("foodPortions")[0]["gramWeight"])
            ing_name.append(name)                        
        else:
            standard_vol.append(np.nan)
            standard_weight.append(np.nan)
            standard_unit.append('')
            ing_name.append(name)                         

           
# ing_name = [item for sublist in ing_name for item in sublist]
ingredient_density['ingredient']= ing_name      
ingredient_density['standard_vol']=standard_vol
ingredient_density['standard_weight_gm'] = standard_weight
ingredient_density['standard_unit'] =standard_unit
ingredient_density_df = pd.DataFrame.from_dict(ingredient_density)

print("time in minutes {}".format((time.time() - start)/60))

time in minutes 87.24851893186569


In [437]:
# ing_dens_comb = ingredient_density_df
ing_dens_comb = pd.concat([ing_dens_comb,ingredient_density_df], ignore_index=True)

In [438]:
ing_dens_comb.shape

(9483, 4)

In [439]:
ing_dens_comb_copy = ing_dens_comb
ing_dens_comb_copy.shape

(9483, 4)

In [419]:
ing_dens_comb.tail()

,ingredient,standard_vol,standard_weight_gm,standard_unit
8857,tablespoon rice wine vinegar,1.00,14.9,tbsp
8858,canned red kidney beans,1.00,266.0,can drained solids
8859,pinch dried parsley,0.25,1.4,cup
8860,low fat sour cream,1.00,240.0,cup
8861,raw cane sugar,1.00,31.0,fl


In [440]:
op_file_path = "./Data/ing_density.csv"
ing_dens_comb.to_csv(op_file_path, index=False)